In [122]:
import numpy as np
import onnx
import onnxruntime as ort
import time
import os
import matplotlib.pyplot as plt
import csv

class Multiply_experiment_class():
    #TODO: Set the input size to the size of a layer in the TINYYOLO 
    def __init__(self):
        self.in1 = np.random.random([1,16,256,256]).astype(np.float32)
        self.b = np.load("b_tensor.npy")
        self.out_ort = np.random.random([1,16,256,256]).astype(np.float32)
        self.expected_res = np.zeros_like(self.out_ort)        
        
        self.ort_runtime=[]
        
    def result_check(self):
        self.expected_res = self.in1*self.b
        return np.allclose(self.out_ort[0],self.expected_res)
        
    def test_ort_mul(self, model_name):
        model_load = onnx.load(model_name)
        input_dict = {"in1": self.in1, "b": self.b}
        sess = ort.InferenceSession(model_load.SerializeToString())
        self.out_ort = sess.run(None,input_dict)           
    
    def throughput_test(self,interations):
        for i in range(interations):
            start = time.time()
            self.test_ort_mul("multiply_model.onnx")
            end = time.time()
            self.ort_runtime.append(end-start)       
        # return self.ort_runtime
    
    def results_publisher(self, filename = "ort_results.csv"):
        print("Interations = %d" %(len(self.ort_runtime)))
        print("Min = %f ms" %(np.mean(self.ort_runtime)*1000))
        print("Max = %f ms" %(np.max(self.ort_runtime)*1000))
        print("Mean = %f ms" %(np.min(self.ort_runtime)*1000))        
        if not os.path.exists("results/"):
            os.makedirs("results/")
        np.savetxt("results/" + filename , np.asarray(self.ort_runtime), delimiter=',')

In [124]:
ort_run_test = Multiply_experiment_class()
ort_run_test.test_ort_mul("multiply_model.onnx")
ort_run_test.throughput_test(100)
if ort_run_test.result_check():
    ort_run_test.results_publisher()

Interations = 100
Min = 5.453775 ms
Max = 7.979393 ms
Mean = 3.944397 ms
